In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
# !unzip /gdrive/My\ Drive/DIRT/full_new_data.zip -d /gdrive/My\ Drive/DIRT/

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 25.0 MB/s eta 0:00:00


In [4]:
!nvidia-smi

Sun Dec  8 00:37:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   42C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
from ultralytics import YOLO
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split
import numpy as np

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
IMAGES_DIR = '/gdrive/My Drive/DIRT/full_new_data/img'  # Путь к вашему датасету с изображениями
MASKS_DIR = '/gdrive/My Drive/DIRT/full_new_data/msk'  # Путь к вашему датасету с масками
OUTPUT_DIR = '/gdrive/My Drive/DIRT/train_data'  # Путь к выходной директории
TRAIN_SIZE = 0.9  # Процент обучающей выборки

In [7]:
os.makedirs(os.path.join(OUTPUT_DIR, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/val'), exist_ok=True)

In [8]:
image_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith(('.jpg', '.png'))]
mask_files = [f for f in os.listdir(MASKS_DIR) if f.endswith('.png')]
if len(image_files) != len(mask_files):
    print(len(image_files))
    print(len(mask_files))
    print("Количество изображений и масок не совпадает.")
    for mask in mask_files:
        if mask.replace(".png", ".jpg") not in image_files:
            print(mask)

In [9]:
train_images, val_images = train_test_split(image_files, train_size=TRAIN_SIZE, random_state=42)

In [10]:
def convert_mask_to_yolo(mask_path, out):
    # Открываем изображение
    image = cv2.imread(mask_path)

    if image is None:
        print(f"Не удалось открыть изображение: {mask_path}")
        return

    height, width = image.shape[:2]

    # Создаем маску для черного цвета
    black_mask = cv2.inRange(image, (0, 0, 0), (50, 50, 50))

    # Создаем новое изображение, где черный цвет остается, а остальные цвета становятся белыми
    new_image = np.ones_like(image) * 255  # Начинаем с белого изображения
    new_image[black_mask > 0] = [0, 0, 0]  # Заменяем черные пиксели

    # Преобразуем в градации серого для нахождения контуров
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Находим контуры
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Сохраняем контуры в текстовом формате
    output_file_name = os.path.splitext(os.path.basename(mask_path))[0] + '.txt'
    output_file_path = os.path.join(OUTPUT_DIR, out, output_file_name)

    with open(output_file_path, 'w') as f:
        for index, contour in enumerate(contours):
            # Получаем координаты всех точек контура
            contour_points = contour.reshape(-1, 2)
            # Нормализуем координаты
            normalized_points = [(x / width, y / height) for x, y in contour_points]
            points_str = ' '.join(f"{x:.3f} {y:.3f}" for x, y in normalized_points)
            f.write(f"0 {points_str}\n")

In [11]:
# Копирование изображений и масок в соответствующие папки
for img in train_images:
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/train', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/train')

In [12]:
for img in val_images:
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/val', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/val')

In [13]:
data_yaml_content = f"""
train: /gdrive/My Drive/DIRT/train_data/images/train
val: /gdrive/My Drive/DIRT/train_data/images/val

nc: 1  # Обновите количество классов (1 для загрязнения)
names: ['contaminated']  # Обновите названия классов
"""

with open('/gdrive/My Drive/DIRT/train_data/data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("Датасет успешно разбит и сохранен в структуре проекта.")

Датасет успешно разбит и сохранен в структуре проекта.


In [10]:
model = YOLO("yolo11n-seg.pt")
# model = YOLO("runs/segment/train5/weights/last.pt")

In [8]:
!pwd

/content


In [11]:
train_results = model.train(
    data="/gdrive/My Drive/DIRT/train_data/data.yaml",  # path to dataset YAML
    epochs=300,  # number of training epochs
    imgsz=256,  # training image size
    device="cuda",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    batch=128,
    # resume=True
)

Ultralytics 8.3.47 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=/gdrive/My Drive/DIRT/train_data/data.yaml, epochs=300, time=None, patience=100, batch=128, imgsz=256, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, s

train: Scanning /gdrive/My Drive/DIRT/train_data/labels/train.cache... 2055 images, 150 backgrounds, 0 corrupt: 100%|██████████| 2055/2055 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /gdrive/My Drive/DIRT/train_data/labels/val.cache... 229 images, 22 backgrounds, 0 corrupt: 100%|██████████| 229/229 [00:00<?, ?it/s]


Plotting labels to runs/segment/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.001), 100 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to runs/segment/train2
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

    291/300      3.76G     0.9633      1.677     0.8033      1.001         25        256: 100%|██████████| 17/17 [00:15<00:00,  1.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.71s/it]

                   all        229        973      0.844      0.732       0.76      0.568      0.867      0.737      0.775      0.468
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 191, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



291 epochs completed in 1.099 hours.
Optimizer stripped from runs/segment/train2/weights/last.pt, 6.0MB
Optimizer stripped from runs/segment/train2/weights/best.pt, 6.0MB

Validating runs/segment/train2/weights/best.pt...
Ultralytics 8.3.47 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.56s/it]


                   all        229        973      0.819      0.773      0.809      0.607       0.86      0.756      0.816      0.499
Speed: 0.0ms preprocess, 0.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/segment/train2


In [ ]:
# results = model.val(data="./train_data/data.yaml")
# print(results)

In [ ]:
def infer_image(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path)

    # Инференс
    return model(image, conf=0.5)

In [ ]:
# Функция для создания маски с черным фоном
def create_mask(image_path, results):
    # Загружаем изображение и переводим в градации серого
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Создаем пустую маску с черным фоном
    mask = np.zeros((height, width), dtype=np.uint8)

    # Проходим по результатам и создаем маску
    for result in results:
        masks = result.masks  # Получаем маски из результатов
        if masks is not None:
            for mask_array in masks.data:  # Получаем маски как массивы
                mask_i = mask_array.cpu().numpy()  # Преобразуем маску в numpy массив

                # Изменяем размер маски под размер оригинального изображения
                mask_i_resized = cv2.resize(mask_i, (width, height), interpolation=cv2.INTER_LINEAR)

                # Накладываем маску на пустую маску (255 для белого)
                mask[mask_i_resized > 0] = 255

    return mask

In [ ]:
results = infer_image("/gdrive/My Drive/DIRT/train_data/images/val/orig_gr1.jpg")
mask_image = create_mask("/gdrive/My Drive/DIRT/train_data/images/val/orig_gr1.jpg", results)

# Сохраняем маску в формате PNG
mask_output_path = './mask_image.png'  # Укажите путь для сохранения маски
cv2.imwrite(mask_output_path, mask_image)


0: 224x384 5 contaminateds, 16.1ms
Speed: 1.7ms preprocess, 16.1ms inference, 4.1ms postprocess per image at shape (1, 3, 224, 384)


True